<a href="https://colab.research.google.com/github/lucas6028/machine_learning/blob/main/CNN/sports_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from google.colab import drive

import os
import numpy as np
import matplotlib.pyplot as plt
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/sports-classification")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9


## Use TensorFlow ImageDataGenerator for Image Loading and Augmentation

Use ImageDataGenerator to preprocess the images (e.g., rescale, rotate, flip) and create generators for training and validation data:

In [16]:
train_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/train'
test_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/test'
val_dir = '/root/.cache/kagglehub/datasets/gpiosenka/sports-classification/versions/9/valid'

# Create ImageDataGenerator instances for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

validation_datagen = ImageDataGenerator(rescale=1./255,)

# Flow training images in batches from the dataset directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' if you have multiple classes
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' if you have multiple classes
)

Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


## Define the CNN Model

In [21]:
model = models.Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer with the number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 100)                 │          51,300 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,447,396 (169.55 MB)

 Trainable params: 44,447,396 (169.55 MB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

Use early stopping to avoid overfitting.

In [22]:
# Define early stopping
early_stopping = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    factor=0.5,
)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping],
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


422/422 ━━━━━━━━━━━━━━━━━━━━ 169s 385ms/step - accuracy: 0.0674 - loss: 4.2173 - val_accuracy: 0.2640 - val_loss: 3.1332 - learning_rate: 0.0010
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 193s 371ms/step - accuracy: 0.3092 - loss: 2.7701 - val_accuracy: 0.3580 - val_loss: 2.6322 - learning_rate: 0.0010
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 204s 377ms/step - accuracy: 0.4261 - loss: 2.2165 - val_accuracy: 0.4360 - val_loss: 2.2822 - learning_rate: 0.0010
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 163s 379ms/step - accuracy: 0.5029 - loss: 1.9040 - val_accuracy: 0.4440 - val_loss: 2.3145 - learning_rate: 0.0010
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 158s 370ms/step - accuracy: 0.5897 - loss: 1.5286 - val_accuracy: 0.4880 - val_loss: 2.1295 - learning_rate: 0.0010
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 203s 371ms/step - accuracy: 0.6487 - loss: 1.2713 - val_accuracy: 0.5020 - val_loss: 2.2860 - learning_rate: 0.0010
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 203s 374ms/step - accuracy: 0.7184 

## Evaluate the model

Once trained, you can evaluate the model on a test set or use it for predictions.

In [23]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5509 - loss: 3.1260
Validation Loss: 3.1622042655944824, Validation Accuracy: 0.5440000295639038


## Store the model

In [ ]:
drive.mount('/content/drive')

# Save TensorFlow model to Google Drive
# model.save('/content/drive/MyDrive/model.h5')

Mounted at /content/drive
